In [1]:
import sys 
sys.path.append('~/Code/Natural_AE/src')

sys.path

['/home/go/Code/Natural_AE/src/model',
 '/usr/lib/python311.zip',
 '/usr/lib/python3.11',
 '/usr/lib/python3.11/lib-dynload',
 '',
 '/home/go/.cache/pypoetry/virtualenvs/natural-adversarial-patch-Pv5n5N3d-py3.11/lib/python3.11/site-packages',
 '~/Code/Natural_AE/src']

In [2]:
import torch
import torch.nn




In [3]:
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class simpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(simpleCNN, self).__init__()
        self.name = "simpleCNN"
        self.conv1 = nn.Conv2d(
            in_channels=3, out_channels=16, kernel_size=3, padding=1
        )
        self.conv2 = nn.Conv2d(
            in_channels=16, out_channels=32, kernel_size=3, padding=1
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
import torch
# モデルのインスタンスを作成
model = simpleCNN(58)

# .pthファイルから重みをロード
weights = torch.load('trained_CNN.pth',torch.device('cpu'))

# モデルに重みを適用
model.load_state_dict(weights)

<All keys matched successfully>

In [138]:
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import csv


class CustomDataset(Dataset):
    def __init__(self, data_dir: Path, is_train: bool = True, size: int = 224):
        """
        Args:
            data_dir (Path): データセットのディレクトリ
                訓練 例 Path("data/traffic_sign/traffic_Data/DATA")
                テスト 例 Path("data/traffic_sign/traffic_Data/TEST")
            is_train (bool, optional): フラグ。 ラベルを得る時に使う。
            size (int, optional): 画像のサイズ。デフォルトは224。
        """
        self.data_dir = data_dir
        self.is_train = is_train
        self.size = size
        # self.unknown_class = ["40", "41", "42", "45", "49", "52", "56", "57"]
        self.unknown_class = []
        self.img_path_list = []
        self.num_classes = 58 - len(self.unknown_class)
        self.int2label = self.get_int2label()

        # 訓練の場合は data_dir/クラス名/ファイル名.png の形式で画像が保存されている
        # unknown_class は除外して、画像のパスを取得する
        if self.is_train:
            for child in self.data_dir.iterdir():
                if (
                    child.is_dir() and child.name not in self.unknown_class
                ):  # unknown_class は除外
                    temp_path_list = [
                        p for p in child.iterdir() if p.suffix == ".png"
                    ]
                    self.img_path_list.extend(temp_path_list)
        # テストの場合は data_dir/ファイル名.png の形式で画像が保存されている
        else:
            for child in self.data_dir.iterdir():
                # unknown_class は除外
                if (
                    child.suffix == ".png"
                    and str(int(child.name[:3])) not in self.unknown_class
                ):
                    self.img_path_list.append(child)

        assert len(self.img_path_list) > 0, "画像が読み込めませんでした。パスを確認してください。"

        # ラベルの数を取得する。

    def custom_transform(self, img):
        """画像の前処理"""
        img = img.resize(
            (self.size, self.size)
        )  # 参照: https://www.kaggle.com/code/boulahchichenadir/cnn-classification
        img = transforms.ToTensor()(img)
        img = transforms.Normalize(
            (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
        )(img)
        return img

    def get_int2label(self):
        int2label = {}
        with open(Path("labels.csv")) as f:
            csv_reader = csv.reader(f)
            next(csv_reader)

            for row in csv_reader:
                key = int(row[0])
                value = row[1]
                int2label[key] = value

        return int2label

    def __len__(self):
        return len(self.img_path_list)

    def __getitem__(self, idx):
        img_path = self.img_path_list[idx]
        img = Image.open(img_path)
        if self.is_train:
            label = int(img_path.parent.name)

        else:
            # ラベル_ファイル名.png 3桁詰めなので整数に直す
            label = int(img_path.name.split("_")[0])
            label = int(label)

        # 最後に処理
        img = self.custom_transform(img)

        return img, label

In [135]:


test_dataset = CustomDataset(
    Path("../traffic_Data/TEST"), is_train=False
)


test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=True)

model = simpleCNN(test_dataset.num_classes)




In [136]:
all = 0 
ok = 0
for images, lbls in test_loader:
    all +=1
    # ここでモデルの学習や推論を行うことができます。
    output = model(images)
    # print(torch.argmax(output))
    # print(lbls)
    if torch.argmax(output)==lbls:
        ok+=1



In [137]:
print(ok/all)

0.01354062186559679


In [140]:
# images = Image.open(Path("../traffic_Data/DATA/0/000_0001.png"))
# images = images.resize((256,256))
output = model(images)

TypeError: conv2d() received an invalid combination of arguments - got (Image, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!Image!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!Image!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
